In [11]:
from flatlib import const
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
#!pip install pyyaml
import yaml, codecs
from flatlib import aspects
import itertools
import math
import pandas as pd
# to fix the installation bug
# pip uninstall pyswisspeh
# pip install pyswisseph==2.00.00-2
# Documentation
# https://flatlib.readthedocs.io/en/latest/tutorials/chart-properties.html


class Zodiac(object):
    STARS = ['Alpheratz', 'Algol', 'Alcyone', 'Alcyone', 'Aldebaran', 'Rigel', 'Capella', 'Betelgeuse', 'Sirius', 'Canopus', 'Castor', 'Pollux', 'Procyon', 'Asellus_Borealis', 'Asellus_Australis', 'Alphard', 'Regulus', 'Denebola', 'Algorab', 'Spica', 'Arcturus', 'Alphecca', 'ZUBEN_ELSCHEMALI', 'Unukalhai', 'Agena', 'RIGEL_CENTAURUS', 'Antares', 'Lesath', 'Vega', 'Altair', 'Deneb_Algedi', 'Fomalhaut', 'DENEB_ADIGE', 'Achernar','Algenib']
    PLANETS = ['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn', 'North_Node', 'South_Node', 'Syzygy', 'Pars_Fortuna']
    all_objects = []
    all_stars = []    
    all_planets = []    
    
    for star in STARS:
        all_stars.append(eval(F"const.STAR_{star.upper()}"))

    for planet in PLANETS:
        all_planets.append(eval(F"const.{planet.upper()}"))
    
    all_objects = all_stars + all_planets
    
    def __init__(self):
        self._parse_signs()
        self._parse_planets()
    
    @staticmethod
    def check_aspects(hrscp):
        major_aspects = []
        minor_aspects = []
        
        for a in Zodiac.all_objects:
            for b in Zodiac.all_objects:
                if a!=b:
                    try:
                        A = hrscp.chart.get(a)
                    except:
                        A = hrscp.chart.getFixedStar(a)
                    try:
                        B = hrscp.chart.get(b)
                    except:
                        B = hrscp.chart.getFixedStar(b)

                    try:
                        aspect = aspects.getAspect(A, B, const.MAJOR_ASPECTS)
                        if aspect.type!=-1:
                            res = ([aspect.active.id, aspect.passive.id], aspect.type,aspect.movement())
                            if res not in major_aspects:
                                major_aspects.append(res)
                    except:
                        pass
                    
                    try:
                        aspect = aspects.getAspect(A, B, const.MINOR_ASPECTS)
                        if aspect.type!=-1:
                            res = ([aspect.active.id, aspect.passive.id], aspect.type,aspect.movement())
                            if res not in major_aspects:
                                minor_aspects.append(res)
                    except:
                        pass
        return {'major_aspects':major_aspects,'minor_aspects': minor_aspects}
    
    def _parse_signs(self):
        f = codecs.open('sign_conditions.yaml','r','utf-8')
        self.signs = yaml.safe_load(f)
        
    def _parse_planets(self):
        f = codecs.open('condition_planets.yml','r','utf-8')
        self.planets = yaml.safe_load(f)
        

class Horoscope(object):
    
    def __init__(self, data='2015/03/13', time='17:00', time_zone='+00:00', geo_lat = '38n32',geo_long= '8w54'):
        self.date = Datetime(data, time, time_zone)
        self.pos = GeoPos(geo_lat, geo_long)
        self.chart = Chart(self.date, self.pos, hsys=const.HOUSES_PLACIDUS)    

    def get_obj(self, name):
        return self.chart.getObject(eval(F"const.{name.upper()}")).__dict__
        
    def get_mars(self):
        return self.chart.getObject(const.MARS).__dict__

    def get_mercury(self):
        return self.chart.getObject(const.MERCURY).__dict__    
    
    def get_venus(self):
        return self.chart.getObject(const.VENUS).__dict__ 

    def get_spica(self):
        return self.chart.getFixedStar('Spica').__dict__
    
    def get_sun(self):
        return self.chart.getObject(const.SUN).__dict__

    def get_moon(self):
        return self.chart.getObject(const.MOON).__dict__

    def get_stars(self):
        '''
            get all stars
        '''        
        stars = dict()
        for idx, star in enumerate(Zodiac.all_stars):
            temp = self.chart.getFixedStar(star).__dict__
            stars[F"{temp['id']}"] = temp

        return stars
    
    def get_houses(self):
        '''
            get all houses
        '''        
        houses = dict()
        for idx, house in enumerate([const.HOUSE1,const.HOUSE2,const.HOUSE2,const.HOUSE4,const.HOUSE5,const.HOUSE6,const.HOUSE7,const.HOUSE8,const.HOUSE9,const.HOUSE10,const.HOUSE11,const.HOUSE12]):
            houses[F'HOUSE_{idx+1}'] = self.chart.get(house).__dict__
            try:
                houses[F'HOUSE_{idx+1}']['condition'] = self.chart.get(house).condition()
            except:
                pass
            try:
                houses[F'HOUSE_{idx+1}']['gender'] = self.chart.get(house).gender()            
            except:
                pass
            
             
            houses[F'HOUSE_{idx+1}']['objects'] = []
            for obj in Zodiac.all_objects:
                try:
                    obj = self.chart.get(obj)
                except:
                    obj = self.chart.getFixedStar(obj)
                if self.chart.get(house).hasObject(obj):                    
                    houses[F'HOUSE_{idx+1}']['objects'].append(obj.id) 

        return houses
    
    def get_objects(self):
        '''
            get all objects
        '''
        objects = dict()
        for obj in self.chart.objects:
            objects[(obj.id)] = obj.__dict__
        return objects
        

    def get_angles(self):
        '''
            get all angles
        '''
        objects = dict()
        for obj in self.chart.angles:
            objects[(obj.id)] = obj.__dict__
        return objects
                
    
    def get_chart_properties(self):
        '''
            get chart properties
        '''
        return {'isDiurnal': self.chart.isDiurnal(),'MoonPhase':self.chart.getMoonPhase()}
    
    def get_aspects(self):
        return Zodiac.check_aspects(self)

import tqdm
from collections import OrderedDict
from datetime import datetime, timedelta

def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta 
        
from math import sin, cos, sqrt, atan2, radians
import numpy as np

def measure_distance(obj1, obj2):

    R = 1

    lat1 = radians(obj1['lat'])
    lon1 = radians(obj1['lon'])
    lat2 = radians(obj2['lat'])
    lon2 = radians(obj2['lon'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

def condition_moon(sun, moon):
    dist = (-sun['lon'] + moon['lon']+360)%360
    return math.ceil((dist/360)*29.5)+2   

def condition_observable_rise(sun, mars):
    return (sun['lon']-mars['lon']) > 4.5 and (sun['lon']-mars['lon']) < 5

def condition_in_virgo(spica, mars, o1, o2):
    conditions = []
    for o in [mars, o1, o2]:
        c1 = spica['lon'] < (o['lon']+35)
        c2 = spica['lon'] > (o['lon']-15)
        c3 = o['lat'] > -10
        c4 = o['lat'] < 20
        conditions.append(all([c1,c2,c3,c4]))
    return all(conditions + [mars['lon'] > o1['lon'], mars['lon'] > o2['lon']])
    

In [30]:
hr = Horoscope(data='-1000/1/1',time='05:30',time_zone=time_zone, geo_lat=geo_lat, geo_long=geo_long)

In [32]:
dts = [(dt.strftime('%Y/%m/%d'),dt.strftime('%H:%M')) for dt in datetime_range(datetime(1, 1, 1), datetime(1, 12, 31), timedelta(minutes=60*24))]
dts

[('0001/01/01', '00:00'),
 ('0001/01/02', '00:00'),
 ('0001/01/03', '00:00'),
 ('0001/01/04', '00:00'),
 ('0001/01/05', '00:00'),
 ('0001/01/06', '00:00'),
 ('0001/01/07', '00:00'),
 ('0001/01/08', '00:00'),
 ('0001/01/09', '00:00'),
 ('0001/01/10', '00:00'),
 ('0001/01/11', '00:00'),
 ('0001/01/12', '00:00'),
 ('0001/01/13', '00:00'),
 ('0001/01/14', '00:00'),
 ('0001/01/15', '00:00'),
 ('0001/01/16', '00:00'),
 ('0001/01/17', '00:00'),
 ('0001/01/18', '00:00'),
 ('0001/01/19', '00:00'),
 ('0001/01/20', '00:00'),
 ('0001/01/21', '00:00'),
 ('0001/01/22', '00:00'),
 ('0001/01/23', '00:00'),
 ('0001/01/24', '00:00'),
 ('0001/01/25', '00:00'),
 ('0001/01/26', '00:00'),
 ('0001/01/27', '00:00'),
 ('0001/01/28', '00:00'),
 ('0001/01/29', '00:00'),
 ('0001/01/30', '00:00'),
 ('0001/01/31', '00:00'),
 ('0001/02/01', '00:00'),
 ('0001/02/02', '00:00'),
 ('0001/02/03', '00:00'),
 ('0001/02/04', '00:00'),
 ('0001/02/05', '00:00'),
 ('0001/02/06', '00:00'),
 ('0001/02/07', '00:00'),
 ('0001/02/0

In [ ]:
#cities = {'Karbala':('32n60','44e02'),'San Fransisco':('37n78','122w42'),'Buenos Aires':('34s60','58w38'), 'Stockholm': ('59n33','18e07') ,'Tehran':('35n69','51e39'),'Bloemfontein':('29s09', '26e16'),'Beijing' :('39n90', '116e41'),'Melbourne': ('37s81', '144e96')}

cities = {'Karbala':('32n60','44e02')}

dts = [(dt.strftime('%Y/%m/%d'),dt.strftime('%H:%M')) for dt in datetime_range(datetime(1, 1, 1), datetime(2032, 12, 1), timedelta(minutes=60*24))]
objects_oi = ['Mercury', 'Venus', 'Jupiter', 'Saturn']

time_zone='+02:56'
results = {'date':[],'city':[],'time':[],'month':[], 'o1':[],'o2':[],'Mars<Spica':[]}

for city, (geo_lat, geo_long) in cities.items():
    
    for d,t in tqdm.tqdm(dts):
        hr = Horoscope(data=d,time='05:30',time_zone=time_zone, geo_lat=geo_lat, geo_long=geo_long)
        mars = hr.get_mars()
        sun = hr.get_sun()
        if condition_observable_rise(sun, mars):
            for o1_name, o2_name in list(itertools.combinations(objects_oi, 2)):
                o1=hr.get_obj(o1_name)
                o2=hr.get_obj(o2_name)
                spica = hr.get_spica()    
                if condition_in_virgo(spica, mars, o1, o2):
                    moon = hr.get_moon()
                    results['city'].append(city)
                    results['date'].append(d)
                    results['time'].append('05:30')
                    results['month'].append(condition_moon(sun, moon))
                    results['o1'].append(o1_name)            
                    results['o2'].append(o2_name)   
                    results['Mars<Spica'].append(mars['lon'] > spica['lon'])

In [23]:
df = pd.DataFrame(results)

In [27]:
pd.DataFrame(results).to_csv('all.csv')

In [24]:
df

,date,city,time,month,o1,o2,Mars<Spica
0,0095/09/10,San Fransisco,05:30,27,Mercury,Saturn,False
1,0300/09/11,San Fransisco,05:30,12,Mercury,Saturn,False
2,0332/09/19,San Fransisco,05:30,14,Mercury,Venus,False
3,0364/09/28,San Fransisco,05:30,17,Mercury,Venus,False
4,0379/09/14,San Fransisco,05:30,18,Mercury,Jupiter,False
...,...,...,...,...,...,...,...
268,1878/10/03,Melbourne,05:30,10,Mercury,Venus,False
269,1910/10/12,Melbourne,05:30,10,Mercury,Venus,False
270,1942/10/20,Melbourne,05:30,13,Mercury,Venus,False
271,1942/10/21,Melbourne,05:30,14,Mercury,Venus,False


In [26]:
df[(df['month']==10)&df['Mars<Spica']]

,date,city,time,month,o1,o2,Mars<Spica
20,0680/10/10,San Fransisco,05:30,10,Mercury,Venus,True
59,0680/10/10,Buenos Aires,05:30,10,Mercury,Venus,True
98,0680/10/10,Stockholm,05:30,10,Mercury,Venus,True
137,0680/10/10,Tehran,05:30,10,Mercury,Venus,True
176,0680/10/10,Bloemfontein,05:30,10,Mercury,Venus,True
215,0680/10/10,Beijing,05:30,10,Mercury,Venus,True
254,0680/10/10,Melbourne,05:30,10,Mercury,Venus,True


In [40]:
#cities = {'Karbala':('32n60','44e02'),'San Fransisco':('37n78','122w42'),'Buenos Aires':('34s60','58w38'), 'Stockholm': ('59n33','18e07') ,'Tehran':('35n69','51e39'),'Bloemfontein':('29s09', '26e16'),'Beijing' :('39n90', '116e41'),'Melbourne': ('37s81', '144e96')}

cities = {'Karbala':('32n60','44e02')}

dts = [(dt.strftime('%Y/%m/%d'),dt.strftime('%H:%M')) for dt in datetime_range(datetime(1, 1, 1), datetime(1, 12, 31), timedelta(minutes=60*24))]
objects_oi = ['Mercury', 'Venus', 'Jupiter', 'Saturn']

geo_lat = '32n60'
geo_long ='44e02'

time_zone='+02:56'
results = {'date':[],'time':[],'month':[], 'o1':[],'o2':[],'Mars<Spica':[]}

for y in tqdm.tqdm(range(-3000,0)):
    for d,t in dts:
        d = '/'.join([str(y)]+d.split('/')[1:3])
        hr = Horoscope(data=d,time='05:30',time_zone=time_zone, geo_lat=geo_lat, geo_long=geo_long)
        mars = hr.get_mars()
        sun = hr.get_sun()
        if condition_observable_rise(sun, mars):
            for o1_name, o2_name in list(itertools.combinations(objects_oi, 2)):
                o1=hr.get_obj(o1_name)
                o2=hr.get_obj(o2_name)
                spica = hr.get_spica()    
                if condition_in_virgo(spica, mars, o1, o2):
                    moon = hr.get_moon()
                    results['date'].append(d)
                    results['time'].append('05:30')
                    results['month'].append(condition_moon(sun, moon))
                    results['o1'].append(o1_name)            
                    results['o2'].append(o2_name)   
                    results['Mars<Spica'].append(mars['lon'] > spica['lon'])

100%|██████████| 3000/3000 [10:47<00:00,  4.63it/s]


In [41]:
df= pd.DataFrame(results)

In [44]:
df.to_csv('negative_years.csv')

In [43]:
df[(df['month']==10)&df['Mars<Spica']]

,date,time,month,o1,o2,Mars<Spica
32,-2602/09/01,05:30,10,Mercury,Venus,True
